In [1]:
import os
from PIL import Image
import torch
from torchvision import transforms


content_dir = r'C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content'
style_dir = r'C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style'

def load_images_from_folder(folder):
    image_paths = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(('jpg', 'png')):
                image_paths.append(os.path.join(root, file))
    return image_paths

content_images = load_images_from_folder(content_dir)
style_images = load_images_from_folder(style_dir)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def load_image(image_path):
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)  
    return image.to(device)

In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.utils import save_image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


vgg = models.vgg19(pretrained=True).features.to(device)
vgg.eval()


for param in vgg.parameters():
    param.requires_grad = False

C:\Users\reeth\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\reeth\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
vgg = models.vgg19(pretrained=True).features.to(device)
vgg.eval()


for param in vgg.parameters():
    param.requires_grad = False


def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '19': 'conv4_1', '28': 'conv5_1'}
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features


def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

In [4]:

def calculate_loss(generated_features, content_features, style_features, style_grams):
    content_loss = torch.mean((generated_features['conv4_1'] - content_features['conv4_1'])**2)
    
    style_loss = 0
    for layer in style_grams:
        generated_gram = gram_matrix(generated_features[layer])
        _, d, h, w = generated_features[layer].shape
        style_loss += torch.mean((generated_gram - style_grams[layer])**2) / (d * h * w)
    
    total_loss = content_loss + (1e5 * style_loss)
    return total_loss


In [5]:
import os
import torch
import torch.optim as optim
from torchvision.utils import save_image


output_dir = r'C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\output'
os.makedirs(output_dir, exist_ok=True)


def style_transfer(content_image, style_image, num_iterations=500, style_weight=1000000, content_weight=1):
    # Initialize target image
    target_image = content_image.clone().requires_grad_(True).to(device)

   
    optimizer = optim.Adam([target_image], lr=0.003)

    
    content_features = get_features(content_image, vgg)
    style_features = get_features(style_image, vgg)
    
   
    style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

    for i in range(num_iterations):
        optimizer.zero_grad()

        target_features = get_features(target_image, vgg)

        
        loss = calculate_loss(target_features, content_features, style_features, style_grams)

        
        loss.backward()
        optimizer.step()

        
        if i % 100 == 0:
            print(f"Iteration {i}, Total loss: {loss.item()}")

    return target_image

for content_path in content_images:
    content_image = load_image(content_path)
    
    for style_path in style_images:
        style_image = load_image(style_path)
        
        
        stylized_image = style_transfer(content_image, style_image)

      
        output_filename = f"{os.path.basename(content_path).split('.')[0]}_styled_with_{os.path.basename(style_path).split('.')[0]}.png"
        save_image(stylized_image.cpu().data, os.path.join(output_dir, output_filename))

print("Style transfer completed for all images!")



Style transfer completed for all images!


In [14]:
import os
from PIL import Image
from torchvision import transforms
import torch


content_folder = r"C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content"  
style_folder = r"C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style"  


content_images = [os.path.join(content_folder, img) for img in os.listdir(content_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]
style_images = [os.path.join(style_folder, img) for img in os.listdir(style_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]


def load_image(image_path, max_size=400, device='cuda' if torch.cuda.is_available() else 'cpu'):
    image = Image.open(image_path).convert('RGB')
    
    
    size = min(max_size, max(image.size))
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    
    image = transform(image).unsqueeze(0).to(device)
    return image


if not content_images:
    print("No content images found in the specified folder.")
if not style_images:
    print("No style images found in the specified folder.")


for content_path in content_images:
    content_image = load_image(content_path)
    
    for style_path in style_images:
        style_image = load_image(style_path)
      
        print(f"Loaded content image from {content_path} and style image from {style_path}")


Loaded content image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content\WhatsApp Image 2024-10-23 at 8.51.57 PM.jpeg and style image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style\WhatsApp Image 2024-10-23 at 8.52.44 PM (1).jpeg
Loaded content image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content\WhatsApp Image 2024-10-23 at 8.51.57 PM.jpeg and style image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style\WhatsApp Image 2024-10-23 at 8.52.44 PM.jpeg
Loaded content image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content\WhatsApp Image 2024-10-23 at 8.51.57 PM.jpeg and style image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style\WhatsApp Image 2024-10-23 at 8.52.45 PM (1).jpeg
Loaded content image from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content\WhatsApp Image 2024-10-23 at 8.51.57 PM.jpe

In [ ]:
import torch
import torch.optim as optim
from torchvision import models


device = 'cuda' if torch.cuda.is_available() else 'cpu'
vgg = models.vgg19(pretrained=True).features.to(device)
vgg.eval()


for param in vgg.parameters():
    param.requires_grad = False


content_weight = 1e4
style_weight = 1e2

def calculate_content_loss(target_features, content_features):
    return torch.mean((target_features - content_features) ** 2)


def calculate_style_loss(target_gram, style_gram):
    return torch.mean((target_gram - style_gram) ** 2)


def run_style_transfer(content_image, style_image, vgg, num_steps=500, style_weight=1e2, content_weight=1e4):
    # Clone the content image as the initial target image
    target_image = content_image.clone().requires_grad_(True).to(device)

    
    content_features = get_features(content_image, vgg)
    style_features = get_features(style_image, vgg)
    style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

    
    optimizer = optim.Adam([target_image], lr=0.003)

    for step in range(1, num_steps + 1):
       
        target_features = get_features(target_image, vgg)
        
        
        content_loss = calculate_content_loss(target_features['conv4_1'], content_features['conv4_1'])
        
        
        style_loss = 0
        for layer in style_grams:
            target_gram = gram_matrix(target_features[layer])
            style_gram = style_grams[layer]
            layer_style_loss = calculate_style_loss(target_gram, style_gram)
            style_loss += layer_style_loss / (target_gram.shape[0] ** 2)

        
        total_loss = content_weight * content_loss + style_weight * style_loss

        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        
        if step % 50 == 0:
            print(f"Step {step}, Total loss: {total_loss.item()}")

    
    return target_image


for content_path in content_images:
    content_image = load_image(content_path)
    
    for style_path in style_images:
        style_image = load_image(style_path)
        print(f"Applying style from {style_path} to content {content_path}...")

        
        output_image = run_style_transfer(content_image, style_image, vgg)

        
        output_image = output_image.cpu().squeeze()
        output_image = transforms.ToPILImage()(output_image)
        
        
        output_image.show()
        output_image.save(f"output_{os.path.basename(content_path)}_{os.path.basename(style_path)}.jpg")


C:\Users\reeth\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\reeth\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Applying style from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style\WhatsApp Image 2024-10-23 at 8.52.44 PM (1).jpeg to content C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content\WhatsApp Image 2024-10-23 at 8.51.57 PM.jpeg...
Step 50, Total loss: 5958148.5
Step 100, Total loss: 3715626.5
Step 150, Total loss: 2645188.75
Step 200, Total loss: 2034617.375
Step 250, Total loss: 1658370.625
Step 300, Total loss: 1410626.75
Step 350, Total loss: 1236284.875
Step 400, Total loss: 1106523.125
Step 450, Total loss: 1005570.5
Step 500, Total loss: 924308.5
Applying style from C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\style\WhatsApp Image 2024-10-23 at 8.52.44 PM.jpeg to content C:\Users\reeth\Projects\Deep Learning-Rotten Fruit Classification\content\WhatsApp Image 2024-10-23 at 8.51.57 PM.jpeg...
Step 50, Total loss: 3914985.75
